### Create interference for LLM

In [1]:
from langgraph.graph import StateGraph, MessagesState
from langgraph.constants import START, END
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import ChatOllama

    
from dotenv import load_dotenv
import os

# ---------------- Load env ----------------
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

checkpoint = InMemorySaver()
store = InMemoryStore()

# ---------------- Models ----------------
gemini_model = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash", api_key=GEMINI_API_KEY
)
ollama_model = ChatOllama(model="gemma3n", disable_streaming=False)

# fallback: if Gemini fails, use Ollama
model = gemini_model.with_fallbacks([ollama_model])

def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    state["messages"].append(response)
    return state


def create_state_graph():
    """
    Creates and returns a state graph with predefined nodes, edges, and memory checkpointing.

    Returns:
        StateGraph: A compiled state graph with nodes and edges set up.
    """

    workflow = (
        StateGraph(MessagesState)
        .add_node("call_model", call_model)
        .add_edge(START, "call_model")
        .add_edge("call_model", END)
        .compile(checkpointer=checkpoint, store=store)
    )

    return workflow

# Create the state graph
resume_agent = create_state_graph()

In [2]:
config = {
    "configurable":{
        "thread_id": "2"
    }
}

In [3]:
for chunk, metadata in resume_agent.stream({"messages":"what is space? explain in long asnwer"}, stream_mode="messages", config=config):
    if chunk.content:
        print(chunk.content, end="\n\n")

Space is one of the most fundamental and profound concepts in physics and philosophy, yet its precise nature remains a subject of ongoing inquiry and debate. Far more than just the "empty" region between objects, our understanding of space has evolved dramatically from

 ancient philosophical ideas to the complex, dynamic entity described by modern physics.

Here's a long explanation of what space is:

---

### The Intuitive and Everyday Understanding

At its most basic, intuitive level, **space is

 the three-dimensional extent in which objects and events have relative position and direction.** It's the "where" of everything. When we say something is "here" or "there," "up" or "down," "left"

 or "right," we are referring to its position in space. It's the container that holds the universe, allowing for movement, separation, and the arrangement of matter. We perceive it as having three dimensions: length, width, and height

, which allow us to define the volume of objects and the dist

### Extract Text from Pdf

In [4]:
from pypdf import PdfReader

def extract_text_from_pdf(file_path: str) -> str:
    """
    Extracts text from a PDF file.
    
    Args:
        file_path (str): Path to the PDF file.
        
    Returns:
        str: Extracted text from all pages.
    """
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""  # Some pages may return None
        text += "\n"
    return text.strip()

In [6]:
pdf_file = r"C:\Users\jigne\Downloads\190210107071_Resume.pdf"
resume_text = extract_text_from_pdf(pdf_file)

In [33]:
SYSTEM_PROMPT = """
You are a resume information extraction agent.  
Your task is to extract structured information from resumes.  

name: full name of the candidate
email: email address
phone: phone number
education: list of education entries, each with degree, institution, start_date, end_date,
grade (if available)
projects: list of projects, each with project_name, description, technologies (list of technologies used),
link (if available)
experience: list of work experience entries, each with job_title, company, location,
start_date, end_date, responsibilities (list of responsibilities/achievements)
skills: list of skills
other_info: dictionary with optional fields:
  certifications: list of certifications (if available)
  languages: list of languages (if available)
  achievements: list of achievements (if available)
  links: dictionary with optional fields linkedin, github, portfolio (if available)

Rules:
- Do not hallucinate. If information is not present, return null or empty list.
- Always return JSON only, with no extra text.
- Keep field values concise but complete.
"""


In [36]:
from pydantic import BaseModel, Field
from typing import List, Optional


class EducationEntry(BaseModel):
    degree: Optional[str] = ""
    institution: Optional[str] = ""
    start_date: Optional[str] = ""
    end_date: Optional[str] = ""
    grade: Optional[str] = ""


class ProjectEntry(BaseModel):
    project_name: Optional[str] = ""
    description: Optional[str] = ""
    technologies: List[str] = Field(default_factory=list)
    link: Optional[str] = ""


class ExperienceEntry(BaseModel):
    job_title: Optional[str] = ""
    company: Optional[str] = ""
    location: Optional[str] = ""
    start_date: Optional[str] = ""
    end_date: Optional[str] = ""
    responsibilities: List[str] = Field(default_factory=list)


class Links(BaseModel):
    linkedin: Optional[str] = ""
    github: Optional[str] = ""
    portfolio: Optional[str] = ""


class OtherInfo(BaseModel):
    certifications: List[str] = Field(default_factory=list)
    languages: List[str] = Field(default_factory=list)
    achievements: List[str] = Field(default_factory=list)
    links: Links = Field(default_factory=Links)


class ResumeExtractionData(BaseModel):
    name: Optional[str] = ""
    email: Optional[str] = ""
    phone: Optional[str] = ""
    education: List[EducationEntry] = Field(default_factory=list)
    projects: List[ProjectEntry] = Field(default_factory=list)
    experience: List[ExperienceEntry] = Field(default_factory=list)
    skills: List[str] = Field(default_factory=list)
    other_info: OtherInfo = Field(default_factory=OtherInfo)


extraction_model = gemini_model.with_structured_output(schema=ResumeExtractionData).with_fallbacks([ollama_model])


In [37]:
from langchain_core.prompts import ChatPromptTemplate


prompt_template = ChatPromptTemplate.from_messages([
    (
        "system", SYSTEM_PROMPT
    ),
    ("human", "Extract all possible information from the following resume text: {text}"),
])


prompt = prompt_template.invoke({"text": resume_text})

In [38]:
response = extraction_model.invoke(prompt)

In [44]:
json_reponse_for_pass_to_llm = response.model_dump_json()

### Create resume templete 

In [104]:
TEMPLATE_PROMPT = f"""
You are a professional resume designer. 
Generate a complete HTML resume with inline CSS styling based only on the JSON data provided. 

- Input will be candidate information in JSON format.  
- Use the JSON data to fill the resume directly. Do not include placeholders.  
- Layout: single-column, ATS-friendly, no tables, no graphics.  
- Typography: Use clean fonts (Arial, Helvetica, sans-serif).  
- Styling: proper spacing, bold section headings, consistent font sizes, clear separation between sections.  
- Dates: Display in format "Start Date - End Date" if available, otherwise omit.  
- Only include sections that have data in the JSON.  
- Output must be valid HTML only (no markdown, no explanations).  

This is the resume data in JSON:
{ response.model_dump }
"""


In [105]:
def generate_resume_template():
    response = gemini_model.invoke(TEMPLATE_PROMPT.format(data=json_reponse_for_pass_to_llm))
    return response

template_html = generate_resume_template()

In [106]:
from jinja2 import Template
from weasyprint import HTML

def render_resume(resume_data, template_html, output_file="resume.pdf"):
    template = Template(template_html)
    html_out = template.render(**resume_data)
    HTML(string=html_out).write_pdf(output_file)
    print(f"✅ Resume generated: {output_file}")

In [107]:
render_resume(response.model_dump(), template_html.content)

✅ Resume generated: resume.pdf
